In [1]:
from __future__ import print_function

''' 
Classify sounds using database
Author: Scott H. Hawley
This is kind of a mixture of Keun Woo Choi's code https://github.com/keunwoochoi/music-auto_tagging-keras
   and the MNIST classifier at https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py
Trained using Fraunhofer IDMT's database of monophonic guitar effects, 
   clips were 2 seconds long, sampled at 44100 Hz
'''

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

from keras.models import Sequential, Model
from keras.layers import Input, Dense, TimeDistributed, LSTM, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.callbacks import ModelCheckpoint
from keras import backend
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
import os
from os.path import isfile

from timeit import default_timer as timer

mono=True

Using TensorFlow backend.


In [2]:
def get_class_names(path="Preproc/"):  # class names are subdirectory names in Preproc/ directory
    class_names = os.listdir(path)
    class_names = class_names[1:]
    return class_names

In [3]:
def file_count(path):
    count = 0
    for files in os.listdir(path):
        if files.endswith('.npy'):
            count = count + 1
    return count
        

In [4]:
def get_total_files(path="Preproc/",train_percentage=0.8): 
    sum_total = 0
    sum_train = 0
    sum_test = 0
    subdirs = os.listdir(path)
    subdirs = subdirs[1:]
    print ("In routine get_total file, subdirs: ", subdirs)
    n_files = 0
    for subdir in subdirs:
        n_files = file_count(path+subdir)       
        sum_total += n_files
        n_train = int(train_percentage*n_files)
        n_test = n_files - n_train
        sum_train += n_train
        sum_test += n_test
    return sum_total, sum_train, sum_test

In [5]:
def get_sample_dimensions(path='Preproc/'):
    classname = os.listdir(path)[1]
    files = os.listdir(path+classname)
    files = files[1:]
    infilename = files[0]
    audio_path = path + classname + '/' + infilename
    melgram = np.load(audio_path)
    print("   get_sample_dimensions: melgram.shape = ",melgram.shape)
    return melgram.shape
 

In [6]:
def encode_class(class_name, class_names):  # makes a "one-hot" vector for each class name called
    try:
        idx = class_names.index(class_name)
        vec = np.zeros(len(class_names))
        vec[idx] = 1
        return vec
    except ValueError:
        return None

In [7]:
def shuffle_XY_paths(X,Y,paths):   # generates a randomized order, keeping X&Y(&paths) together
    assert (X.shape[0] == Y.shape[0] )
    idx = np.array(range(Y.shape[0]))
    np.random.shuffle(idx)
    newX = np.copy(X)
    newY = np.copy(Y)
    newpaths = paths
    for i in range(len(idx)):
        newX[i] = X[idx[i],:,:]
        newY[i] = Y[idx[i],:]
        newpaths[i] = paths[idx[i]]
    return newX, newY, newpaths

In [8]:
def get_class_files(path):
    class_files = os.listdir(path)
    try:
        class_files.remove('.DS_Store')
    except ValueError:
        pass
    return class_files

In [9]:
#def onehot(x):
#    return to_categorical(x)

In [13]:
'''
So we make the training & testing datasets here, and we do it separately.
Why not just make one big dataset, shuffle, and then split into train & test?
because we want to make sure statistics in training & testing are as similar as possible
'''
def build_datasets(train_percentage=0.8, preproc=False):
    if (preproc):
        path = "Preproc/"
    else:
        path = "Samples/"

    class_names = get_class_names(path=path)
    print("class_names = ",class_names)

    total_files, total_train, total_test = get_total_files(path=path, train_percentage=train_percentage)
    print("total files = ",total_files)

    nb_classes = len(class_names)

    # pre-allocate memory for speed (old method used np.concatenate, slow)
    mel_dims = get_sample_dimensions(path=path)  # Find out the 'shape' of each data file
    X_train = np.zeros((total_train, mel_dims[1], mel_dims[2], mel_dims[3]))   
    Y_train = np.zeros((total_train, nb_classes))  
    X_test = np.zeros((total_test, mel_dims[1], mel_dims[2], mel_dims[3]))  
    Y_test = np.zeros((total_test, nb_classes))  
    paths_train = []
    paths_test = []

    train_count = 0
    test_count = 0
    for idx, classname in enumerate(class_names):
        this_Y = np.array(encode_class(classname,class_names) )
        this_Y = this_Y[np.newaxis,:]
        class_files = get_class_files(path+classname)
        n_files = len(class_files)
        n_load =  n_files
        n_train = int(train_percentage * n_load)
        printevery = 5
        print("")
        for idx2, infilename in enumerate(class_files[0:n_load]):          
            audio_path = path + classname + '/' + infilename
            if (0 == idx2 % printevery):
                print('\r Loading class: {:14s} ({:2d} of {:2d} classes)'.format(classname,idx+1,nb_classes),
                       ", file ",idx2+1," of ",n_load,": ",audio_path,sep="")
            #start = timer()
            if (preproc):
                melgram = np.load(audio_path)
#              sr = 44100
                sr = 22050
            else:
              aud, sr = librosa.load(audio_path, mono=mono,sr=None)
              melgram = librosa.logamplitude(librosa.feature.melspectrogram(aud, sr=sr, n_mels=96),ref_power=1.0)[np.newaxis,np.newaxis,:,:]

            melgram = melgram[:,:,:,0:mel_dims[3]]   # just in case files are differnt sizes: clip to first file size
       
            #end = timer()
            #print("time = ",end - start) 
            if (idx2 < n_train):
                # concatenate is SLOW for big datasets; use pre-allocated instead
                #X_train = np.concatenate((X_train, melgram), axis=0)  
                #Y_train = np.concatenate((Y_train, this_Y), axis=0)
                X_train[train_count,:,:] = melgram
                Y_train[train_count,:] = this_Y
                paths_train.append(audio_path)     # list-appending is still fast. (??)
                train_count += 1
            else:
                X_test[test_count,:,:] = melgram
                Y_test[test_count,:] = this_Y
                #X_test = np.concatenate((X_test, melgram), axis=0)
                #Y_test = np.concatenate((Y_test, this_Y), axis=0)
                paths_test.append(audio_path)
                test_count += 1
        print("")

    print("Shuffling order of data...")
    X_train, Y_train, paths_train = shuffle_XY_paths(X_train, Y_train, paths_train)
    X_test, Y_test, paths_test = shuffle_XY_paths(X_test, Y_test, paths_test)

    return X_train, Y_train, paths_train, X_test, Y_test, paths_test, class_names, sr

In [14]:
def build_model(X,Y,nb_classes):
    nb_filters = 4  # number of convolutional filters to use
    pool_size = (2, 2)  # size of pooling area for max pooling
    kernel_size = (3, 3)  # convolution kernel size
    nb_layers = 4
    input_shape = (1, X.shape[2], X.shape[3])

    model = Sequential()
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid', input_shape=input_shape))
    model.add(BatchNormalization(axis=1, mode=2))
    model.add(Activation('relu'))

    for layer in range(nb_layers-1):
        model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
        model.add(BatchNormalization(axis=1, mode=2))
        model.add(ELU(alpha=1.0))  
        model.add(MaxPooling2D(pool_size=pool_size))
        model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation("softmax"))
    return model
    

In [15]:
if __name__ == '__main__':
    np.random.seed(1)

    # get the data
    X_train, Y_train, paths_train, X_test, Y_test, paths_test, class_names, sr = build_datasets(preproc=True)

class_names =  ['flute', 'tabla']
In routine get_total file, subdirs:  ['flute', 'tabla']
total files =  46
   get_sample_dimensions: melgram.shape =  (1, 1, 96, 431)

 Loading class: flute          ( 1 of  2 classes), file 1 of 23: Preproc/flute/f10.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 6 of 23: Preproc/flute/f16.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 11 of 23: Preproc/flute/f20.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 16 of 23: Preproc/flute/f3.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 21 of 23: Preproc/flute/f7.wav.npy


 Loading class: tabla          ( 2 of  2 classes), file 1 of 23: Preproc/tabla/L0.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 6 of 23: Preproc/tabla/L13.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 11 of 23: Preproc/tabla/L18.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 16 of 23: Preproc/tabla/L22.wav.npy
 Loading 

In [24]:
x, sr = librosa.load("f2.wav")
np.shape(x)

(110250,)

In [30]:
    # make the model
    model = build_model(X_train,Y_train, nb_classes=len(class_names))
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    model.summary()

    # Initialize weights using checkpoint if it exists. (Checkpointing requires h5py)
    load_checkpoint = True
    checkpoint_filepath = 'weights.hdf5'
    if (load_checkpoint):
        print("Looking for previous weights...")
        if ( isfile(checkpoint_filepath) ):
            print ('Checkpoint file detected. Loading weights.')
            model.load_weights(checkpoint_filepath)
        else:
            print ('No checkpoint file detected.  Starting from scratch.')
    else:
        print('Starting from scratch (no checkpoint)')
    checkpointer = ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True)


    # train and score the model
    batch_size = 2
    nb_epoch = 10
    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test), callbacks=[checkpointer])
    score = model.evaluate(X_test, Y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

class_names =  ['flute', 'tabla']
In routine get_total file, subdirs:  ['flute', 'tabla']
total files =  46
   get_sample_dimensions: melgram.shape =  (1, 1, 96, 431)

 Loading class: flute          ( 1 of  2 classes), file 1 of 23: Preproc/flute/f10.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 6 of 23: Preproc/flute/f16.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 11 of 23: Preproc/flute/f20.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 16 of 23: Preproc/flute/f3.wav.npy
 Loading class: flute          ( 1 of  2 classes), file 21 of 23: Preproc/flute/f7.wav.npy


 Loading class: tabla          ( 2 of  2 classes), file 1 of 23: Preproc/tabla/L0.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 6 of 23: Preproc/tabla/L13.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 11 of 23: Preproc/tabla/L18.wav.npy
 Loading class: tabla          ( 2 of  2 classes), file 16 of 23: Preproc/tabla/L22.wav.npy
 Loading 

/Users/ardhendupathak/anaconda2/envs/ForMusic/lib/python2.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), padding="valid", input_shape=(1, 96, 43...)`
  # Remove the CWD from sys.path while we load stuff.


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_4/convolution' (op: 'Conv2D') with input shapes: [?,1,96,431], [3,3,431,4].

In [15]:
np.shape(X_train)

(36, 1, 96, 431)

In [16]:
np.shape(Y_train)

(36, 2)

In [17]:
np.shape(X_test)

(10, 1, 96, 431)

In [18]:
np.shape(Y_test)

(10, 2)

In [25]:
aud, sr = librosa.load("/Users/ardhendupathak/NN-Keras/MusicProject/NBs/Samples/flute/f2.wav")
sr

22050